In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import urllib
import os
import sys
import random
import glob
import re
import time
from io import StringIO
import bs4

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from cleaner import *

In [2]:
docs = glob.glob("../data/raw/*.pdf")

In [32]:
t = docs[0]
t

'../data/raw\\1Aprto30Jun2014_DCMS_clean.pdf'

In [33]:
"""
See:
- https://euske.github.io/pdfminer/programming.html
- http://denis.papathanasiou.org/posts/2010.08.04.post.html
"""

import pdfminer

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice

from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator

laparams = LAParams()

fp = open(t, 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)

if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

rsrcmgr = PDFResourceManager()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

layouts = []
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    layouts.append(device.get_result())

In [34]:
l = layouts[6]

In [36]:
texts = []
rects = []
other = []

for n, e in enumerate(l):
    if isinstance(e, pdfminer.layout.LTTextBoxHorizontal):
        texts.append(e)
    elif isinstance(e, pdfminer.layout.LTRect):
        rects.append(e)
    else:
        other.append(e)

In [37]:
characters = []


Department for Culture, Media and Sport 

Department for Culture, Media and Sport 

Department for Culture, Media and Sport 



In [64]:
TEXT_ELEMENTS = [
    pdfminer.layout.LTTextBox,
    pdfminer.layout.LTTextBoxHorizontal,
    pdfminer.layout.LTTextLine,
    pdfminer.layout.LTTextLineHorizontal
]

def extract_characters(element):
    if isinstance(element, pdfminer.layout.LTChar):
        return [element]
    
    if any(isinstance(element, i) for i in TEXT_ELEMENTS):
        elements = []
        for e in element:
            elements += extract_characters(e)
        return elements
    return []

In [71]:
texts[0].get_text()

u'Department for Culture, Media and Sport \n'

In [78]:
for t in texts:
    print("------")
    print(t.get_text())
    print("".join([c.get_text() for c in extract_characters(t)]))

------
Department for Culture, Media and Sport 

Department for Culture, Media and Sport 
------
 

 
------
 

 
------
MEETINGS WITH EXTERNAL ORGANISATIONS2 (INCLUDING 

MEETINGS WITH EXTERNAL ORGANISATIONS2 (INCLUDING 
------
MEETINGS WITH NEWSPAPER AND OTHER MEDIA PROPRIETORS, 

MEETINGS WITH NEWSPAPER AND OTHER MEDIA PROPRIETORS, 
------
EDITORS AND SENIOR EXECUTIVES)  

EDITORS AND SENIOR EXECUTIVES)  
------
Secretary of State, Rt Hon Maria Miller MP (to 8 April) 
Date of Meeting   Name of Organisation 
2 April 2014 

Secretary of State, Rt Hon Maria Miller MP (to 8 April) Date of Meeting  Name of Organisation 2 April 2014 
------
Hay Group 

Hay Group 
------
Secretary of State, Rt Hon Sajid Javid MP (from 9 April) 
Date of Meeting   Name of Organisation 
29 April 2014 
7 May 2014 
8 May 2014 

Secretary of State, Rt Hon Sajid Javid MP (from 9 April) Date of Meeting  Name of Organisation 29 April 2014 7 May 2014 8 May 2014 
------
BBC 
BskyB, Jeremy Darroch 
Michael Burleigh 

